# Multimodal Mountain Peak Search — Colab Quickstart

This notebook shows how to:
- Install deps
- Connect to **Elasticsearch**
- Create indices
- Index a small **peaks catalog** (text+image blended vectors)
- Index a few **photos**
- Run **text → image** search and **identify-from-photo** search

> Minimal demo for the LinkedIn/blog post. No Streamlit here; it’s pure Python cells so anyone can run it in Colab.


In [ ]:
#@title Install dependencies
!pip -q install --upgrade pip
!pip -q install elasticsearch pillow pillow-heif PyYAML transformers huggingface-hub \
                torch torchvision folium streamlit-folium
print("✅ Installed")

In [ ]:
#@title Configure Elasticsearch (API key recommended)
import os, base64

# 👉 Set ONE of these:
ES_URL = os.environ.get("ES_URL", "http://localhost:9200")      # if you have a local ES tunnel
ES_CLOUD_ID = os.environ.get("ES_CLOUD_ID", "")                 # if using Elastic Cloud

# Option A: single base64 api key (id:key base64-encoded)
ES_API_KEY_B64 = os.environ.get("ES_API_KEY_B64", "")

# Option B: id + key (assemble and base64-encode)
ES_API_KEY_ID  = os.environ.get("ES_API_KEY_ID", "")
ES_API_KEY     = os.environ.get("ES_API_KEY", "")

if not ES_API_KEY_B64 and (ES_API_KEY_ID and ES_API_KEY):
    ES_API_KEY_B64 = base64.b64encode(f"{ES_API_KEY_ID}:{ES_API_KEY}".encode()).decode()

print("ES_URL     :", ES_URL)
print("CLOUD_ID?  :", bool(ES_CLOUD_ID))
print("API_KEY_B64:", "set" if ES_API_KEY_B64 else "MISSING")

# Propagate for scripts
os.environ["ES_URL"] = ES_URL
os.environ["ES_CLOUD_ID"] = ES_CLOUD_ID
os.environ["ES_API_KEY_B64"] = ES_API_KEY_B64

# Optional model override
# os.environ["SIGLIP_MODEL_ID"] = "google/siglip-so400m-patch14-384"


In [ ]:
#@title Clone your repo (set the URL first!)
REPO_URL = "https://github.com/navneet83/multimodal-mountain-peak-search"  # <-- CHANGE THIS
TARGET_DIR = "/content/multimodal-mountain-peak-search"
import os, shutil, subprocess, sys

if os.path.exists(TARGET_DIR):
    shutil.rmtree(TARGET_DIR)

print("Cloning:", REPO_URL)
rc = subprocess.call(["git","clone","--depth","1", REPO_URL, TARGET_DIR])
if rc != 0:
    raise SystemExit("❌ Clone failed. Set REPO_URL to your GitHub repo.")

os.chdir(TARGET_DIR)
sys.path.insert(0, os.path.join(TARGET_DIR, "src"))  # import ai_mpi.embeddings
print("✅ Cloned and cwd set to", TARGET_DIR)

In [ ]:
#@title Create data folders + write a tiny peaks.yaml
import os, yaml

os.makedirs("data/peaks", exist_ok=True)
os.makedirs("data/images", exist_ok=True)

peaks = [
    {"id":"ama-dablam","names":["Ama Dablam"],"latlon":{"lat":27.86167,"lon":86.86139},"elev_m":6812},
    {"id":"mount-everest","names":["Mount Everest","Sagarmatha","Chomolungma","Qomolangma"],"latlon":{"lat":27.98833,"lon":86.92528},"elev_m":8849},
    {"id":"pumori","names":["Pumori"],"latlon":{"lat":28.01556,"lon":86.82861},"elev_m":7161},
]
with open("data/peaks.yaml","w") as f:
    yaml.safe_dump(peaks, f, sort_keys=False)

print("✅ Wrote data/peaks.yaml with", len(peaks), "peaks")

In [ ]:
#@title (Optional) Add 1–3 reference images per peak (improves accuracy)
# Upload small reference photos; place them under data/peaks/<id>/
# You can skip this cell; the demo works with text-only prototypes too.
from google.colab import files
import os, shutil

for p in ["ama-dablam","mount-everest","pumori"]:
    os.makedirs(f"data/peaks/{p}", exist_ok=True)

print("📤 Select a couple of tiny JPG/PNG files (optional).")
uploads = files.upload()
for name in uploads.keys():
    shutil.move(name, f"data/peaks/ama-dablam/{name}")

!find data/peaks -maxdepth 2 -type f -print

In [ ]:
#@title Create indices in Elasticsearch
!python scripts/create_indices.py --recreate || python scripts/create_indices.py
print("✅ Indices ready")

In [ ]:
#@title Index peaks (blended text + reference images)
!python scripts/embed_and_index_photos.py --index-peaks --peaks-yaml data/peaks.yaml --peaks-images-root data/peaks --blend-alpha-text 0.55 --blend-max-images 3
print("✅ Peaks indexed")

In [ ]:
#@title Upload a few of your photos to index
from google.colab import files
import shutil, os

print("📤 Upload 3–10 JPG/PNG/HEIC images (keeps the demo snappy).")
uploads = files.upload()
for name in uploads.keys():
    shutil.move(name, f"data/images/{name}")

!find data/images -maxdepth 1 -type f -print

In [ ]:
#@title Index your photos
!python scripts/embed_and_index_photos.py --index-photos --images data/images --topk-predicted 5
print("✅ Photos indexed")

In [ ]:
#@title Text → image search (type a peak name)
from elasticsearch import Elasticsearch
from ai_mpi.embeddings import Siglip2
import numpy as np
from IPython.display import display, HTML

def es_client():
    cloud_id = os.getenv("ES_CLOUD_ID","")
    url      = os.getenv("ES_URL","http://localhost:9200")
    api_b64  = os.getenv("ES_API_KEY_B64","")
    if cloud_id:
        return Elasticsearch(cloud_id=cloud_id, api_key=api_b64 if api_b64 else None)
    return Elasticsearch(url, api_key=api_b64 if api_b64 else None)

es = es_client()
emb = Siglip2()
PHOTOS_INDEX = os.getenv("PHOTOS_INDEX","photos")

def l2norm(v): 
    import numpy as np
    v = np.asarray(v, dtype=np.float32)
    return v/(np.linalg.norm(v)+1e-12)

def prompt_vec(peak_name: str):
    prompts = [
        f"a natural photo of the mountain peak {peak_name} in the Himalayas, Nepal",
        f"{peak_name} landmark peak in the Khumbu region, alpine landscape",
        f"{peak_name} mountain summit, snow, rocky ridgeline",
    ]
    proto = sum([emb.text_vec(p) for p in prompts]) / 3.0
    anti  = emb.text_vec("painting, illustration, poster, map, logo")
    return l2norm(proto - 0.25*anti).astype("float32")

query = "Ama Dablam"  #@param ["Ama Dablam", "Pumori", "Mount Everest"] {allow-input: true}
k = 12  #@param {type:"slider", min:6, max:30, step:2}

qvec = prompt_vec(query)
resp = es.search(index=PHOTOS_INDEX, body={
    "knn": {"field":"clip_image", "query_vector": qvec.tolist(), "k": int(k), "num_candidates": 1000},
    "_source": ["path","predicted_peaks","clip_image","shot_time","gps"]
})
hits = resp.get("hits",{}).get("hits",[])

print(f"Top {len(hits)} results for “{query}”")
cards = []
for h in hits:
    s = h.get("_source",{})
    score = h.get("_score", 0.0)
    path = s.get("path")
    preds = ", ".join(s.get("predicted_peaks", [])[:2])
    ts = (s.get("shot_time") or "").split("T")[0]
    cards.append(f"<div style='display:inline-block;margin:6px;text-align:center'>"
                 f"<img src='data/images/{path}' width='220'/><br>"
                 f"<div style='font-size:12px;color:#888'>knn {score:.3f} | {preds} | {ts}</div></div>")
display(HTML("".join(cards)))

In [ ]:
#@title Identify from photo → similar photos
from google.colab import files
from PIL import Image
import numpy as np
from IPython.display import display, HTML

PEAKS_INDEX = os.getenv("PEAKS_INDEX","peaks_catalog")

print("📤 Upload ONE image to identify")
uploads = files.upload()
img_path = list(uploads.keys())[0]

im = Image.open(img_path).convert("RGB")
ivec = emb.image_vec(im).astype("float32")

# Step 1: image → nearest peaks
resp = es.search(index=PEAKS_INDEX, body={
    "knn": {"field":"text_embed", "query_vector": ivec.tolist(), "k": 3, "num_candidates": 500},
    "_source": ["id","names","text_embed"]
})
hits = resp.get("hits",{}).get("hits",[])
if not hits:
    raise SystemExit("No peak guesses; did you index peaks_catalog?")
best = hits[0]["_source"]
best_name = (best.get("names") or [best.get("id")])[0]
print("Top guess:", best_name)

# Step 2: use the best peak name → text vector → photos kNN
def l2norm(v): 
    v = np.asarray(v, dtype=np.float32)
    return v/(np.linalg.norm(v)+1e-12)

def prompt_vec(peak_name: str):
    prompts = [
        f"a natural photo of the mountain peak {peak_name} in the Himalayas, Nepal",
        f"{peak_name} landmark peak in the Khumbu region, alpine landscape",
        f"{peak_name} mountain summit, snow, rocky ridgeline",
    ]
    proto = np.mean([emb.text_vec(p) for p in prompts], axis=0)
    anti  = emb.text_vec("painting, illustration, poster, map, logo")
    return l2norm(proto - 0.25*anti).astype("float32")

qvec = prompt_vec(best_name)
resp2 = es.search(index=PHOTOS_INDEX, body={
    "knn": {"field":"clip_image", "query_vector": qvec.tolist(), "k": 12, "num_candidates": 1000},
    "_source": ["path","predicted_peaks","clip_image","shot_time","gps"]
})
hits2 = resp2.get("hits",{}).get("hits",[])

cards = [f"<div style='margin:6px 0;font-weight:600;'>Similar photos for “{best_name}”</div>"]
for h in hits2:
    s = h.get("_source",{})
    score = h.get("_score", 0.0)
    path = s.get("path")
    preds = ", ".join(s.get("predicted_peaks", [])[:2])
    ts = (s.get("shot_time") or "").split("T")[0]
    cards.append(f"<div style='display:inline-block;margin:6px;text-align:center'>"
                 f"<img src='data/images/{path}' width='220'/><br>"
                 f"<div style='font-size:12px;color:#888'>knn {score:.3f} | {preds} | {ts}</div></div>")
display(HTML("".join(cards)))